In [1]:
import pandas as pd
from datetime import datetime, timedelta


In [2]:
inventory_df = pd.read_excel('data/New Inventory.xlsx')
receive_df = pd.read_excel('data/Inventory Dataset.xlsx')


In [10]:
receive_df.head()

,Item Number,Date,Inventory
0,064-01-1347,2023-06-17,3561
1,064-01-1361,2023-06-17,4035
2,064-01-1375,2023-06-17,2684
3,064-01-1385,2023-06-17,5136
4,064-01-1399,2023-06-17,6399


In [9]:
inventory_df.head()

,Item Number,Inventory Receive,Total Item Qty
0,064-13-0002,2023-07-07,700
1,064-13-0338,2023-07-07,1700
2,064-13-0340,2023-07-07,1000
3,064-13-0353,2023-07-07,1900
4,064-13-0416,2023-07-07,1000


In [3]:
merged_df = pd.merge(inventory_df, receive_df, on='Item Number', how='outer')
merged_df['Date'] = merged_df.apply(lambda row: row['Date'] if pd.notnull(row['Date']) else row['Inventory Receive'], axis=1)
merged_df['Inventory'].fillna(0, inplace=True)
merged_df['Total Item Qty'].fillna(0, inplace=True)


In [4]:
pivot_table = merged_df.pivot_table(index='Item Number', columns='Date', values='Inventory', aggfunc='sum', fill_value=0)
pivot_table.columns = pd.to_datetime(pivot_table.columns)


In [6]:
max_date = max(merged_df['Date'])
weekly_dates = [max_date + timedelta(days=7 * i) for i in range(1, 12)]


In [11]:
weekly_dates

[Timestamp('2023-12-22 00:00:00'),
 Timestamp('2023-12-29 00:00:00'),
 Timestamp('2024-01-05 00:00:00'),
 Timestamp('2024-01-12 00:00:00'),
 Timestamp('2024-01-19 00:00:00'),
 Timestamp('2024-01-26 00:00:00'),
 Timestamp('2024-02-02 00:00:00'),
 Timestamp('2024-02-09 00:00:00'),
 Timestamp('2024-02-16 00:00:00'),
 Timestamp('2024-02-23 00:00:00'),
 Timestamp('2024-03-01 00:00:00')]

In [7]:
for date in weekly_dates:
    pivot_table[date] = 0
pivot_table = pivot_table.reindex(sorted(pivot_table.columns), axis=1)
for item_number, row in pivot_table.iterrows():
    current_inventory = row.iloc[0]
    for i, receive_date in enumerate(row.index[1:], start=1):
        if i <= len(weekly_dates):
            weekly_date = weekly_dates[i - 1]
            filtered_data = merged_df[(merged_df['Item Number'] == item_number) & (merged_df['Date'] == receive_date)]
            total_item_qty = filtered_data['Total Item Qty']
            if not total_item_qty.empty:
                current_inventory += total_item_qty.values[0]
            pivot_table.at[item_number, weekly_date] = current_inventory

In [12]:
pivot_table

Date,2023-06-17,2023-08-04,2023-08-16,2023-09-08,2023-09-15,2023-09-22,2023-09-24,2023-09-29,2023-10-08,2023-10-13,...,2023-12-29,2024-01-05,2024-01-12,2024-01-19,2024-01-26,2024-02-02,2024-02-09,2024-02-16,2024-02-23,2024-03-01
Item Number,,,,,,,,,,,,,,,,,,,,,
064-01-0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5488,5488,7352,7352,7352
064-01-0005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7380,7380,9900,9900,9900
064-01-0006,10,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
064-01-0012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6840,6840,10084,10084,10084
064-01-0013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6300,6300,8048,8048,8048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
064-18-9884,12419,0,0,0,0,0,0,0,0,0,...,12419,12419,12419,12419,12419,12419,12419,12419,12419,12419
064-18-9904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
064-18-9909,12,0,0,0,0,0,0,0,0,0,...,12,12,12,12,12,12,12,12,12,12
